In [46]:
# ----------------------------------------------------------------------------
#
# TITLE - Make_tables.ipynb
# AUTHOR - James Lane
# PROJECT - OHStars
#
# ----------------------------------------------------------------------------
#
# Docstrings and metadata:
'''
Make the tables including information for all the stars.
'''

__author__ = "James Lane"

In [47]:
### Imports

## Basic
import numpy as np
import sys, os, pdb, warnings
from tqdm import tqdm_notebook as tqdm_nb

## Plotting
import matplotlib as mpl
from matplotlib import pyplot as plt
from matplotlib import colors
from matplotlib import cm
from matplotlib import patches

## Astropy
from astropy import table
from astropy import units as apu
from astropy.utils.exceptions import AstropyWarning

## galpy
from galpy import orbit
from galpy import potential
from galpy import actionAngle
from galpy.util import bovy_coords as gpcoords
from galpy.util import bovy_conversion as gpconv
from galpy.util import bovy_plot as gpplot

# Project-specific
sys.path.append('../../src')
import ohstars.analysis

In [48]:
gaia_filename = '../../data/gaia/SB02_results.vot'
sb02_filename = '../4-SB02_parameters/M17_orbit_parameters.FIT'
rv_filename = '../../data/gaia/SB02_RV.csv'

In [54]:
with warnings.catch_warnings():
    warnings.simplefilter('ignore', AstropyWarning)
    gaia_data = table.Table.read(gaia_filename)
###w
sb02_data = table.Table.read(sb02_filename)
rv_data = table.Table.read(rv_filename, format='csv')
star_names = rv_data['star_name']

assert len(sb02_data) == len(gaia_data) and len(sb02_data) == len(rv_data)

n_stars = len(sb02_data)

In [55]:
sb02_data.colnames

['Name',
 'rp',
 'rperi_err',
 'rapo',
 'rapo_err',
 'Lz',
 'Lz_err',
 'Tr',
 'Tr_err',
 'Eccen',
 'Eccen_err',
 'Zmax',
 'Zmax_err',
 'Energy',
 'Energy_err',
 'Jr',
 'Jr_err',
 'Jphi',
 'Jphi_err',
 'Jz',
 'Jz_err',
 'U',
 'U_err',
 'V',
 'V_err',
 'W',
 'W_err']

In [51]:
# rv_data.colnames

In [52]:
# gaia_data.colnames

In [73]:
# List the stars studied by Monty
M19_stars = ['G037-037','G158-100','G184-007','G189-050','G233-026','G262-021']

# List the properties to include in the table
output_properties = ['rp','rapo','Tr','Eccen','Zmax','U','W','V']
round_digits = [2,2,1,2,2,2,2,2]

n_output_properties = len(output_properties)

In [74]:
# Make the table
fileout = open('table_B1.txt','w')

fileout.write('Name & '+\
              '$r_{peri}$ [kpc] & '+\
              '$r_{apo}$ [kpc] & '+\
              '$T_{rad}$ [Myr] & '+\
              '$e$ &'+\
              '$Z_{max}$ [kpc] &'+\
              'U [km/s] &'+\
              'V [km/s] &'+\
              'W [km/s] &'+\
              '\\\ \n')
fileout.write('\hline \n')

for i in range(len(M19_stars)):
    
    # Find the correct index
    this_ind = np.where(star_names == M19_stars[i])[0][0]
    this_row = sb02_data[this_ind]
    
    fileout.write(star_names[this_ind])
    
    for j in range(n_output_properties):
        
        this_prop = output_properties[j]
        this_prop_err = this_prop+'_err'
        
        if this_prop_err == 'rp_err':
            this_prop_err = 'rperi_err'
        ##fi
        
        prop_val = this_row[this_prop]
        prop_err = this_row[this_prop_err]
    
        if this_prop == 'Tr':
            prop_val *= 1000
            prop_err *= 1000
        ##fi
    
        write_prop_val = str(round(prop_val,round_digits[j]))
        write_prop_err = str(round(prop_err,round_digits[j]))
        
        while_counter = 1
        while write_prop_err == '0.0':
            write_prop_val = str(round(prop_val,round_digits[j]+while_counter))
            write_prop_err = str(round(prop_err,round_digits[j]+while_counter))
            while_counter += 1
        ##fi
        
        fileout.write(' & $' + write_prop_val +\
                      ' \pm ' + write_prop_err + \
                      '$'
                     )
    ###j
    
    fileout.write(' \\\ \n')
    
###i

fileout.write('\n \hline \n\n')
    
    
for i in range(n_stars - len(M19_stars)):
    
    # Find the correct index
    if star_names[i] in M19_stars:
        continue
    this_row = sb02_data[i]
    
    fileout.write(star_names[i])
    
    for j in range(n_output_properties):
        
        this_prop = output_properties[j]
        this_prop_err = this_prop+'_err'
        
        if this_prop_err == 'rp_err':
            this_prop_err = 'rperi_err'
        ##fi
        
        prop_val = this_row[this_prop]
        prop_err = this_row[this_prop_err]
        
        if this_prop == 'Tr':
            prop_val *= 1000
            prop_err *= 1000
        ##fi
        
        write_prop_val = str(round(prop_val,round_digits[j]))
        write_prop_err = str(round(prop_err,round_digits[j]))
        
        while_counter = 1
        while write_prop_err == '0.0':
            write_prop_val = str(round(prop_val,round_digits[j]+while_counter))
            write_prop_err = str(round(prop_err,round_digits[j]+while_counter))
            while_counter += 1
        ##fi
        
        fileout.write(' & $' + write_prop_val +\
                      ' \pm ' + write_prop_err + \
                      '$'
                     )
    ###j
    
    fileout.write(' \\\ \n')
    
###i

fileout.write('\n \hline \n')

fileout.close()

In [75]:
# List the stars studied by Monty
M19_stars = ['G037-037','G158-100','G184-007','G189-050','G233-026','G262-021']

# List the properties to include in the table
output_properties = ['Lz','Jr','Jz']
round_digits = [2,2,2]

n_output_properties = len(output_properties)

In [76]:
# Make the table
fileout = open('table_B2.txt','w')

fileout.write('Name & '+\
              '$L_{z}$ [kpc km/s] & '+\
              '$J_{r}$ [kpc km/s] & '+\
              '$J_{z}$ [kpc km/s] & '+\
              '\\\ \n')
fileout.write('\hline \n')

for i in range(len(M19_stars)):
    
    # Find the correct index
    this_ind = np.where(star_names == M19_stars[i])[0][0]
    this_row = sb02_data[this_ind]
    
    fileout.write(star_names[this_ind])
    
    for j in range(n_output_properties):
        
        this_prop = output_properties[j]
        this_prop_err = this_prop+'_err'
        
        if this_prop_err == 'rp_err':
            this_prop_err = 'rperi_err'
        ##fi
        
        prop_val = this_row[this_prop]
        prop_err = this_row[this_prop_err]
        
        write_prop_val = str(round(prop_val,round_digits[j]))
        write_prop_err = str(round(prop_err,round_digits[j]))
        
        while_counter = 1
        while write_prop_err == '0.0':
            write_prop_val = str(round(prop_val,round_digits[j]+while_counter))
            write_prop_err = str(round(prop_err,round_digits[j]+while_counter))
            while_counter += 1
        ##fi
        
        fileout.write(' & $' + write_prop_val +\
                      ' \pm ' + write_prop_err + \
                      '$'
                     )
    ###j
    
    fileout.write(' \\\ \n')
    
###i

fileout.write('\n \hline \n\n')
    
    
for i in range(n_stars - len(M19_stars)):
    
    # Find the correct index
    if star_names[i] in M19_stars:
        continue
    this_row = sb02_data[i]
    
    fileout.write(star_names[i])
    
    for j in range(n_output_properties):
        
        this_prop = output_properties[j]
        this_prop_err = this_prop+'_err'
        
        if this_prop_err == 'rp_err':
            this_prop_err = 'rperi_err'
        ##fi
        
        prop_val = this_row[this_prop]
        prop_err = this_row[this_prop_err]
        
        write_prop_val = str(round(prop_val,round_digits[j]))
        write_prop_err = str(round(prop_err,round_digits[j]))
        
        while_counter = 1
        while write_prop_err == '0.0':
            write_prop_val = str(round(prop_val,round_digits[j]+while_counter))
            write_prop_err = str(round(prop_err,round_digits[j]+while_counter))
            while_counter += 1
        ##fi
        
        fileout.write(' & $' + write_prop_val +\
                      ' \pm ' + write_prop_err + \
                      '$'
                     )
    ###j
    
    fileout.write(' \\\ \n')
    
###i

fileout.write('\n \hline \n')

fileout.close()